In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import keras_tuner as kt

In [ ]:
INPUT_SHAPE = (224, 224, 3)
DATA_DIR = './datasets/dmd/labels'
BATCH_SIZE = 32
IMG_SIZE = (180, 180)
VALIDATION_SPLIT = 0.2

In [ ]:
(train_ds, val_ds) = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels='inferred',
    label_mode='int',
    validation_split=VALIDATION_SPLIT,
    subset='both',
    color_mode='grayscale',
    seed=2,
    shuffle=0,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)

In [ ]:
layers_data_augmentation = tf.keras.Sequential([
        preprocessing.RandomFlip("horizontal"),
        preprocessing.RandomRotation(0.3, fill_mode='constant'),
        preprocessing.RandomZoom(0.2),
        tf.keras.layers.RandomBrightness(factor=0.2)]
)

In [ ]:
def model_builder(hp):
    model = tf.keras.Sequential();
    # model.add(layers_data_augmentation)
    model.add(tf.keras.layers.Flatten(input_shape=IMG_SIZE))

    # hp_layer_1_nodes = hp.Int('layer_1_nodes', min_value=32, max_value=128, step=16)
    # hp_layer_2_nodes = hp.Int('layer_2_nodes', min_value=32, max_value=128, step=16)
    # hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    hp_layer_1_nodes = 80
    hp_layer_2_nodes = 128
    hp_learning_rate = 1e-2

    model.add(tf.keras.layers.Dense(units=hp_layer_1_nodes, activation='relu'))
    model.add(tf.keras.layers.Dense(units=hp_layer_2_nodes, activation='relu'))
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

    model.compile(
        # optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=hp_learning_rate),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )
    return model

In [ ]:
tuner = kt.Hyperband(model_builder,
     objective='accuracy',  # duda: deberíamos de utilizar 'val_accuracy' (validación) pero no funciona.
     max_epochs=2,
     factor=3,
     directory='models',
     project_name='cnn_hp'
     )

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
tuner.search(train_ds, epochs=2, callbacks=[early_stop])

In [ ]:
best_hps= tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]
loss, accuracy = best_model.evaluate(val_ds)
print("Accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hp)

In [ ]:
best_model_update = tf.keras.callbacks.ModelCheckpoint('./models/cnn/best', save_best_only=True, monitor='accuracy')
model.fit(train_ds, batch_size=BATCH_SIZE, epochs=50,  callbacks=[early_stop, best_model_update])

In [ ]:
print(model.summary())